In [1]:
import tensorflow as tf  # TensorFlow library for building and deploying machine learning models
import pandas as pd  # Pandas library for data manipulation and analysis
import requests  # Requests library for making HTTP requests
import json  # JSON library for parsing and handling JSON data
import base64  # Base64 library for encoding and decoding binary data to/from ASCII
import random  # Random library for generating random numbers and performing random operations

In [2]:
import pandas as pd
import random

# Load the CSV data into a DataFrame
df = pd.read_csv('data/data.csv')

# Remove the 'stroke' column from the DataFrame
df.pop('stroke')

# Get the column names from the DataFrame
columns = df.columns.values

# Generate a random index to select a row from the DataFrame
random_index = random.randint(0, len(df) - 1)  # Note: use len(df) - 1 to stay within bounds

# Retrieve the feature values for the randomly selected row
features = df.values[random_index]

# Create a dictionary where the keys are column names and the values are the feature values of the selected row
inputs = {key: value for key, value in zip(columns, features)}

# Convert the dictionary to a DataFrame
inputs_df = pd.DataFrame([inputs])

# Print the DataFrame containing the feature values for the selected row
inputs_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,45048,Female,21.0,0,0,No,Private,Urban,134.45,29.1,never smoked


In [3]:
def int_feature(value):
    """Create a TensorFlow Feature from an integer value.
    
    Args:
        value (int): The integer value to convert.
        
    Returns:
        tf.train.Feature: TensorFlow Feature object.
    """
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value])
    )

def float_feature(value):
    """Create a TensorFlow Feature from a float value.
    
    Args:
        value (float): The float value to convert.
        
    Returns:
        tf.train.Feature: TensorFlow Feature object.
    """
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value])
    )

def string_feature(value):
    """Create a TensorFlow Feature from a string value.
    
    Args:
        value (str): The string value to convert.
        
    Returns:
        tf.train.Feature: TensorFlow Feature object.
    """
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.encode('utf-8')])
    )

def prepare_json(inputs: dict) -> str:
    """Prepare JSON for TensorFlow Serving with base64-encoded TFRecord examples.
    
    Args:
        inputs (dict): A dictionary with feature names and values.
        
    Returns:
        str: JSON string formatted for TensorFlow Serving.
    """
    # Create a dictionary to hold TensorFlow Features
    feature_spec = {}

    # Convert each input value to the corresponding TensorFlow Feature
    for key, value in inputs.items():
        if isinstance(value, int):
            feature_spec[key] = int_feature(value)
        elif isinstance(value, float):
            feature_spec[key] = float_feature(value)
        elif isinstance(value, str):
            feature_spec[key] = string_feature(value)

    # Create a TensorFlow Example and serialize it to a string
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec))
    example_serialized = example.SerializeToString()

    # Encode the serialized Example to base64 and format it as JSON
    result = [{
        'examples': {
            'b64': base64.b64encode(example_serialized).decode('utf-8')
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [9]:
import requests
import json

def predict(inputs):
    """Send a request to the TensorFlow Serving model for predictions.
    
    Args:
        inputs (dict): A dictionary of feature names and values for prediction.
        
    Returns:
        float: The predicted probability value.
    """
    # Send a POST request to the TensorFlow Serving endpoint with prepared JSON data
    response = requests.post(
        'http://localhost:8080/v1/models/stroke-detection-model:predict',
        data=prepare_json(inputs),
        headers={'Content-Type': 'application/json'}  # Specify content type as JSON
    )

    # Check if the request was successful
    response.raise_for_status()
    
    # Extract the prediction from the response
    predictions = response.json().get('predictions', [[0]])
    predict = predictions[0][0]
    
    return predict

# Get the prediction
prediction = predict(inputs)

# Print the input features and prediction result

print(f'Prediction - {"Detected: Stroke" if prediction > 0.6 else "Detected: Not Stroke"}')

Prediction - Detected: Not Stroke
